In [264]:
import pandas as pd
import numpy as np
df = pd.read_csv('dct.csv')
df=df.drop(['customerID'], axis=1)
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


## cleaning ##

In [265]:
df = pd.get_dummies(df, columns=['InternetService', 'PaymentMethod', 'Contract'], drop_first=True, dtype=int)

In [266]:

import pandas as pd
binary_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'PaperlessBilling', 'Churn', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies' ]


for col in binary_cols:
    df[col] = df[col].str.lower().str.strip()

binary_map = {
    'yes': 1, 
    'no': 0,
    'female': 0,
    'male': 1,
    'True': 1,
    'False': 0,
    'no phone service': 0,
    'no internet service': 0
}

df[binary_cols] = df[binary_cols].replace(binary_map)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(subset=['TotalCharges'], inplace=True)


C:\Users\Dell\AppData\Local\Temp\ipykernel_33380\324721216.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[binary_cols] = df[binary_cols].replace(binary_map)


## feature engineering ##

In [267]:
df['Senior_TechSupport'] = df['SeniorCitizen'] * df['TechSupport']
df['Senior_TechSupport'] = df['Senior_TechSupport'].fillna(0)

df['Fiber_Streaming'] = df['InternetService_Fiber optic'] * df['StreamingTV']
df['Fiber_Streaming'] = df['Fiber_Streaming'].fillna(0)

df['Tenure_Contract'] = df['tenure'] * df['Contract_Two year']
df['Tenure_Contract'] = df['Tenure_Contract'].fillna(0)

df['HasMultipleLines'] = df['PhoneService'] * df['MultipleLines']
df['HasMultipleLines'] = df['HasMultipleLines'].fillna(0)

df['AutoPay_Risk'] = df['PaperlessBilling'] * df['PaymentMethod_Electronic check']
df['AutoPay_Risk'] = df['AutoPay_Risk'].fillna(0)

df['Contract_OneYr_MonthlyCharges'] = df['MonthlyCharges'] * df['Contract_One year']
df['Contract_OneYr_MonthlyCharges'] = df['Contract_OneYr_MonthlyCharges'].fillna(0)

df['Contract_TwoYr_MonthlyCharges'] = df['MonthlyCharges'] * df['Contract_Two year']
df['Contract_TwoYr_MonthlyCharges'] = df['Contract_TwoYr_MonthlyCharges'].fillna(0)

In [268]:
X = df.drop(columns='Churn')
y = df['Churn']


## trainning and testing ##

In [269]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
acc = accuracy_score(y_test, y_pred)
print(f" Accuracy: {acc * 100:.2f} %")





[[926 107]
 [191 183]]
              precision    recall  f1-score   support

           0       0.83      0.90      0.86      1033
           1       0.63      0.49      0.55       374

    accuracy                           0.79      1407
   macro avg       0.73      0.69      0.71      1407
weighted avg       0.78      0.79      0.78      1407

 Accuracy: 78.82 %
